# Manage Conversation History

One of the most common use cases for persistence is to use it to keep track of conversation history. This is great - it makes it easy to continue conversations. As conversations get longer and longer, however, this conversation history can build up and take up more and more of the context window. This can often be undesirable as it leads to more expensive and longer calls to the LLM, and potentially ones that error. In order to prevent this from happening, you need to properly manage the conversation history.

---

Message history can grow quickly and exceed LLM context window size, whether you're building chatbots with many conversation turns or agentic systems with numerous tool calls. There are several strategies for managing the message history:

* [message trimming](#keep-the-original-message-history-unmodified) — remove first or last N messages in the history
* [summarization](#summarizing-message-history) — summarize earlier messages in the history and replace them with a summary
* custom strategies (e.g., message filtering, etc.)

---

Long conversations can exceed the LLM's context window. Common solutions are:

* [Summarization](#summarize-message-history): Maintain a running summary of the conversation
* [Trimming](#trim-message-history): Remove first or last N messages in the history

This allows the agent to keep track of the conversation without exceeding the LLM's context window.

## Setup

First, let's set up the packages we're going to want to use

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_anthropic

Next, we need to set API keys for Anthropic (the LLM we will use)

In [36]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")

ANTHROPIC_API_KEY:  ········


<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>

## Processing Messages

### Use inside a node

In [ ]:
To process message in

In [ ]:
def process_messages(messages: list[AnyMessage]):
    # keep only the last message
    return messages[-1:]
    
def call_model(state: MessagesState):
    # highlight-next-line
    messages = filter_messages(state["messages"])
    response = model.invoke(messages)
    return {"messages": [response]}


builder = StateGraph(MessagesState)
builder.add_node(call_model)
...

??? example "Full example: process messages in `call_model` node"

    ```python
    from langchain.chat_models import init_chat_model
    from langchain_core.messages import AnyMessage
    
    from langgraph.graph import MessagesState, StateGraph, START
    from langgraph.checkpoint.memory import InMemorySaver
    
    def process_messages(messages: list[AnyMessage]):
        # keep only the last message
        return messages[-1:]
    
    def call_model(state: MessagesState):
        # highlight-next-line
        messages = process_messages(state["messages"])
        response = model.invoke(messages)
        return {"messages": [response]}
    
    
    # highight-next-line
    checkpointer = InMemorySaver()
    model = init_chat_model("anthropic:claude-3-5-haiku-latest")
    
    builder = StateGraph(MessagesState)
    builder.add_node(call_model)
    builder.add_edge(START, "call_model")
    # highight-next-line
    app = builder.compile(checkpointer=checkpointer)
    
    config = {"configurable": {"thread_id": "1"}}
    for event in app.stream(
        {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
        config,
        stream_mode="values"
    ):
        event["messages"][-1].pretty_print()
    
    # This will now not remember the previous messages
    # because we set `messages[-1:]` in the process_messages function
    for event in app.stream(
        {"messages": [{"role": "user", "content": "what's my name?"}]},
        config,
        stream_mode="values"
    ):
        event["messages"][-1].pretty_print()
    ```

In [ ]:
# hide-cell
from langchain.chat_models import init_chat_model
from langchain_core.messages import AnyMessage

from langgraph.graph import MessagesState, StateGraph, START
from langgraph.checkpoint.memory import InMemorySaver

def process_messages(messages: list[AnyMessage]):
    # keep only the last message
    return messages[-1:]

def call_model(state: MessagesState):
    # highlight-next-line
    messages = process_messages(state["messages"])
    response = model.invoke(messages)
    return {"messages": [response]}


# highight-next-line
checkpointer = InMemorySaver()
model = init_chat_model("anthropic:claude-3-5-haiku-latest")

builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_edge(START, "call_model")
# highight-next-line
app = builder.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "1"}}
for event in app.stream(
    {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
    config,
    stream_mode="values"
):
    event["messages"][-1].pretty_print()

# This will now not remember the previous messages
# because we set `messages[-1:]` in the process_messages function
for event in app.stream(
    {"messages": [{"role": "user", "content": "what's my name?"}]},
    config,
    stream_mode="values"
):
    event["messages"][-1].pretty_print()

### Use as a separate node

Use this if you need to preprocess inputs to the LLM **and** update the graph state. This is useful when for summarizing message history, where `process_messages` step needs to return updated messages and running summary information.

In [ ]:
from typing_extensions import TypedDict

class OverallState(MessagesState):
    some_key: str

class LLMInputState(TypedDict):
    # highlight-next-line
    llm_input_messages: list[AnyMessage]

def process_messages(state: MessagesState):
    return {
        # keep only the last message
        "llm_input_messages": state["messages"][-1:],  # (1)!
        "some_key": "some_value"  # (2)!
    }

# highlight-next-line
def call_model(state: LLMInputState):  # (3)!
    response = model.invoke(state["llm_input_messages"])
    return {"messages": response}

builder = StateGraph(OverallState)

1. `llm_input_messages` will be visible only to the `call_model` node and won't be added to overall graph state.
2. `some_key` will be added to the **overall** graph state.
3. `call_model` uses a private input schema, different from the `OverallState`. This allows us to change the input to `call_model` node without affecting the **overall** graph state.

??? example "Full example: process messages in a separate node"

    ```python
    from typing_extensions import TypedDict
    
    class OverallState(MessagesState):
        some_key: str
    
    
    class LLMInputState(TypedDict):
        # highlight-next-line
        llm_input_messages: list[AnyMessage]
    
    
    def process_messages(state: MessagesState):
        return {
            # keep only the last message
            "llm_input_messages": state["messages"][-1:],  # (1)!
            "some_key": "some_value"  # (2)!
        }
    
    
    # highlight-next-line
    def call_model(state: LLMInputState):  # (3)!
        response = model.invoke(state["llm_input_messages"])
        return {"messages": response}
    
    
    checkpointer = InMemorySaver()
    model = init_chat_model("anthropic:claude-3-5-haiku-latest")
    
    builder = StateGraph(OverallState)
    # highlight-next-line
    builder.add_node(process_messages)
    builder.add_node(call_model)
    builder.add_edge(START, "process_messages")
    builder.add_edge("process_messages", "call_model")
    
    app = builder.compile(checkpointer=checkpointer)
    
    config = {"configurable": {"thread_id": "1"}}
    for event in app.stream(
        {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
        config,
        stream_mode="updates"
    ):
        print(event)
        print("\n")
    
    # This will now not remember the previous messages
    # because we set `messages[-1:]` in the filter_messages function
    for event in app.stream(
        {"messages": [{"role": "user", "content": "what's my name?"}]},
        config,
        stream_mode="updates"
    ):
        print(event)
        print("\n")
    ```
    
    1. `llm_input_messages` will be visible only to the `call_model` node and won't be added to overall graph state.
    2. `some_key` will be added to the **overall** graph state.
    3. `call_model` uses a private input schema, different from the `OverallState`. This allows us to change the input to `call_model` node without affecting the **overall** graph state.

In [95]:
# hide-cell
from typing_extensions import TypedDict

class OverallState(MessagesState):
    some_key: str


class LLMInputState(TypedDict):
    # highlight-next-line
    llm_input_messages: list[AnyMessage]


def process_messages(state: MessagesState):
    return {
        # keep only the last message
        "llm_input_messages": state["messages"][-1:],
        "some_key": "some_value"
    }


# highlight-next-line
def call_model(state: LLMInputState):
    response = model.invoke(state["llm_input_messages"])
    return {"messages": response}


checkpointer = InMemorySaver()
model = init_chat_model("anthropic:claude-3-5-haiku-latest")

builder = StateGraph(OverallState)
# highlight-next-line
builder.add_node(process_messages)
builder.add_node(call_model)
builder.add_edge(START, "process_messages")
builder.add_edge("process_messages", "call_model")

app = builder.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "1"}}
for event in app.stream(
    {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
    config,
    stream_mode="updates"
):
    print(event)
    print("\n")

# This will now not remember the previous messages
# because we set `messages[-1:]` in the filter_messages function
for event in app.stream(
    {"messages": [{"role": "user", "content": "what's my name?"}]},
    config,
    stream_mode="updates"
):
    print(event)
    print("\n")

{'process_messages': {'llm_input_messages': [HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}, id='6eb8c38a-7f7c-4c7e-98ab-2e01e3e860a9')], 'some_key': 'some_value'}}


{'call_model': {'messages': AIMessage(content='Hi Bob! How are you doing today? Is there anything I can help you with?', additional_kwargs={}, response_metadata={'id': 'msg_01SoZtnvMLsSJCQdL6ajXFK8', 'model': 'claude-3-5-haiku-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 12, 'output_tokens': 21}, 'model_name': 'claude-3-5-haiku-20241022'}, id='run-d8998b94-4f49-494b-9b91-85f676cba22a-0', usage_metadata={'input_tokens': 12, 'output_tokens': 21, 'total_tokens': 33, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})}}


{'process_messages': {'llm_input_messages': [HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}, id='9a38c63e-f23d-4c38-a06d-7d

## Strategies

### Filter messages

The most straight-forward thing to do to prevent conversation history from blowing up is to filter the list of messages before they get passed to the LLM. This involves two parts: defining a function to filter messages, and then adding it to the graph. See the example below which defines a really simple `filter_messages` function and then uses it.

### Trim messages

---

### Delete messages

One of the common states for a graph is a list of messages. Usually you only add messages to that state. However, sometimes you may want to remove messages (either by directly modifying the state or as part of the graph). To do that, you can use the `RemoveMessage` modifier. In this guide, we will cover how to do that.

The key idea is that each state key has a `reducer` key. This key specifies how to combine updates to the state. The default `MessagesState` has a messages key, and the reducer for that key accepts these `RemoveMessage` modifiers. That reducer then uses these `RemoveMessage` to delete messages from the key.

So note that just because your graph state has a key that is a list of messages, it doesn't mean that that this `RemoveMessage` modifier will work. You also have to have a `reducer` defined that knows how to work with this.

!!! important "Valid message history"

    Many models expect certain rules around lists of messages. For example, some expect them to start with a `user` message, others expect all messages with tool calls to be followed by a tool message. **When deleting messages, you will want to make sure you don't violate these rules.**

### Outside the graph

First, we will cover how to manually delete messages. Let's take a look at the current state of the thread:

In [9]:
messages = app.get_state(config).values["messages"]
messages

[HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}, id='db576005-3a60-4b3b-8925-dc602ac1c571'),
 AIMessage(content="It's nice to meet you, Bob! I'm an AI assistant created by Anthropic. I'm here to help out with any questions or tasks you might have. Please let me know if there's anything I can assist you with.", additional_kwargs={}, response_metadata={'id': 'msg_01BKAnYxmoC6bQ9PpCuHk8ZT', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 52}}, id='run-3a60c536-b207-4c56-98f3-03f94d49a9e4-0', usage_metadata={'input_tokens': 12, 'output_tokens': 52, 'total_tokens': 64}),
 HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={}, id='2088c465-400b-430b-ad80-fad47dc1f2d6'),
 AIMessage(content='You said your name is Bob.', additional_kwargs={}, response_metadata={'id': 'msg_013UWTLTzwZi81vke8mMQ2KP', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'e

We can call `update_state` and pass in the id of the first message. This will delete that message.

In [10]:
from langchain_core.messages import RemoveMessage

app.update_state(config, {"messages": RemoveMessage(id=messages[0].id)})

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1ef75157-f251-6a2a-8005-82a86a6593a0'}}

If we now look at the messages, we can verify that the first one was deleted.

In [8]:
messages = app.get_state(config).values["messages"]
messages

[AIMessage(content="It's nice to meet you, Bob! I'm Claude, an AI assistant created by Anthropic. How can I assist you today?", response_metadata={'id': 'msg_01XPSAenmSqK8rX2WgPZHfz7', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 32}}, id='run-1c69af09-adb1-412d-9010-2456e5a555fb-0', usage_metadata={'input_tokens': 12, 'output_tokens': 32, 'total_tokens': 44}),
 HumanMessage(content="what's my name?", id='f3c71afe-8ce2-4ed0-991e-65021f03b0a5'),
 AIMessage(content='Your name is Bob, as you introduced yourself at the beginning of our conversation.', response_metadata={'id': 'msg_01BPZdwsjuMAbC1YAkqawXaF', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 52, 'output_tokens': 19}}, id='run-b2eb9137-2f4e-446f-95f5-3d5f621a2cf8-0', usage_metadata={'input_tokens': 52, 'output_tokens': 19, 'total_tokens': 71})]

### Inside the graph

We can also delete messages programmatically from inside the graph. Here we'll modify the graph to delete any old messages (longer than 3 messages ago) at the end of a graph run.

#### TODO: should the delete messages live before or after call model??

In [9]:
# highlight-next-line
from langchain_core.messages import RemoveMessage

def delete_messages(state):
    messages = state["messages"]
    if len(messages) > 3:
        # highlight-next-line
        return {"messages": [RemoveMessage(id=m.id) for m in messages[:-3]]}

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

builder = StateGraph(MessagesState)
builder.add_sequence([delete_messages, call_model])
builder.add_edge(START, "call_model")
graph = builder.compile(checkpointer=checkpointer)

We can now try this out. We can call the graph twice and then check the state

In [10]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="hi! I'm bob")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    print([(message.type, message.content) for message in event["messages"]])


input_message = HumanMessage(content="what's my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    print([(message.type, message.content) for message in event["messages"]])

[('human', "hi! I'm bob")]
[('human', "hi! I'm bob"), ('ai', "Hello Bob! It's nice to meet you. I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you might have. Please let me know how I can assist you.")]
[('human', "hi! I'm bob"), ('ai', "Hello Bob! It's nice to meet you. I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you might have. Please let me know how I can assist you."), ('human', "what's my name?")]
[('human', "hi! I'm bob"), ('ai', "Hello Bob! It's nice to meet you. I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you might have. Please let me know how I can assist you."), ('human', "what's my name?"), ('ai', 'You said your name is Bob, so that is the name I have for you.')]
[('ai', "Hello Bob! It's nice to meet you. I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you might have. Please let me know how I can assist you."), (

If we now check the state, we should see that it is only three messages long. This is because we just deleted the earlier messages - otherwise it would be four!

In [11]:
messages = app.get_state(config).values["messages"]
messages

[AIMessage(content="Hello Bob! It's nice to meet you. I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you might have. Please let me know how I can assist you.", response_metadata={'id': 'msg_01XPEgPPbcnz5BbGWUDWTmzG', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 48}}, id='run-eded3820-b6a9-4d66-9210-03ca41787ce6-0', usage_metadata={'input_tokens': 12, 'output_tokens': 48, 'total_tokens': 60}),
 HumanMessage(content="what's my name?", id='a0ea2097-3280-402b-92e1-67177b807ae8'),
 AIMessage(content='You said your name is Bob, so that is the name I have for you.', response_metadata={'id': 'msg_01JGT62pxhrhN4SykZ57CSjW', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 68, 'output_tokens': 20}}, id='run-ace3519c-81f8-45fe-a777-91f42d48b3a3-0', usage_metadata={'input_tokens': 68, 'output_tokens': 20, 'tot

!!! warning "Valid message history"

    Remember, when deleting messages you will want to make sure that the remaining message list is still valid. This message list **may actually not be** - this is because it currently starts with an AI message, which some models do not allow.

---

## Summarize messages

One of the most common use cases for persistence is to use it to keep track of conversation history. This is great - it makes it easy to continue conversations. As conversations get longer and longer, however, this conversation history can build up and take up more and more of the context window. This can often be undesirable as it leads to more expensive and longer calls to the LLM, and potentially ones that error. One way to work around that is to create a summary of the conversation to date, and use that with the past N messages. This guide will go through an example of how to do that.

This will involve a few steps:

- Check if the conversation is too long (can be done by checking number of messages or length of messages)
- If yes, the create summary (will need a prompt for this)
- Then remove all except the last N messages

A big part of this is deleting old messages. For an in depth guide on how to do that, see [this guide](../delete-messages)

### TODO: replace this completely (use helper?)

## Build the chatbot

Let's now build the chatbot.

In [25]:
from typing import Literal

from langchain_anthropic import ChatAnthropic
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END

memory = MemorySaver()


# We will add a `summary` attribute (in addition to `messages` key,
# which MessagesState already has)
class State(MessagesState):
    summary: str


# We will use this model for both the conversation and the summarization
model = ChatAnthropic(model_name="claude-3-haiku-20240307")


# Define the logic to call the model
def call_model(state: State):
    # If a summary exists, we add this in as a system message
    summary = state.get("summary", "")
    if summary:
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# We now define the logic for determining whether to end or summarize the conversation
def should_continue(state: State) -> Literal["summarize_conversation", END]:
    """Return the next node to execute."""
    messages = state["messages"]
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
    # Otherwise we can just end
    return END


def summarize_conversation(state: State):
    # First, we summarize the conversation
    summary = state.get("summary", "")
    if summary:
        # If a summary already exists, we use a different system prompt
        # to summarize it than if one didn't
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
    else:
        summary_message = "Create a summary of the conversation above:"

    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    # We now need to delete messages that we no longer want to show up
    # I will delete all but the last two messages, but you can change this
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}


# Define a new graph
workflow = StateGraph(State)

# Define the conversation node and the summarize node
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `conversation`.
    # This means these are the edges taken after the `conversation` node is called.
    "conversation",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `summarize_conversation` to END.
# This means that after `summarize_conversation` is called, we end.
workflow.add_edge("summarize_conversation", END)

# Finally, we compile it!
app = workflow.compile(checkpointer=memory)

## Using the graph

In [26]:
def print_update(update):
    for k, v in update.items():
        for m in v["messages"]:
            m.pretty_print()
        if "summary" in v:
            print(v["summary"])

In [27]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="hi! I'm bob")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="what's my name?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="i like the celtics!")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

It's nice to meet you, Bob! I'm an AI assistant created by Anthropic. How can I help you today?
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is Bob, as you told me at the beginning of our conversation.
================================ Human Message =================================

i like the celtics!
================================== Ai Message ==================================

That's great, the Celtics are a fun team to follow! Basketball is an exciting sport. Do you have a favorite Celtics player or a favorite moment from a Celtics game you've watched? I'd be happy to discuss the team and the sport with you.


We can see that so far no summarization has happened - this is because there are only six messages in the list.

In [28]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content="hi! I'm bob", id='6534853d-b8a7-44b9-837b-eb7abaf7ebf7'),
  AIMessage(content="It's nice to meet you, Bob! I'm an AI assistant created by Anthropic. How can I help you today?", response_metadata={'id': 'msg_015wCFew2vwMQJcpUh2VZ5ah', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 12, 'output_tokens': 30}}, id='run-0d33008b-1094-4f5e-94ce-293283fc3024-0'),
  HumanMessage(content="what's my name?", id='0a4f203a-b95a-42a9-b1c5-bb20f68b3251'),
  AIMessage(content='Your name is Bob, as you told me at the beginning of our conversation.', response_metadata={'id': 'msg_01PLp8wg2xDsJbNR9uCtxcGz', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 50, 'output_tokens': 19}}, id='run-3815dd4d-ee0c-4fc2-9889-f6dd40325961-0'),
  HumanMessage(content='i like the celtics!', id='ac128172-42d1-4390-b7cc-7bcb2d22ee48'),
  AIMessage(content="That

Now let's send another message in

In [29]:
input_message = HumanMessage(content="i like how much they win")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

i like how much they win
================================== Ai Message ==================================

That's understandable, the Celtics have been one of the more successful NBA franchises over the years. Their history of winning championships is very impressive. It's always fun to follow a team that regularly competes for titles. What do you think has been the key to the Celtics' sustained success? Is there a particular era or team that stands out as your favorite?
================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


===========================

If we check the state now, we can see that we have a summary of the conversation, as well as the last two messages

In [30]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='i like how much they win', id='bb916ce7-534c-4d48-9f92-e269f9dc4859'),
  AIMessage(content="That's understandable, the Celtics have been one of the more successful NBA franchises over the years. Their history of winning championships is very impressive. It's always fun to follow a team that regularly competes for titles. What do you think has been the key to the Celtics' sustained success? Is there a particular era or team that stands out as your favorite?", response_metadata={'id': 'msg_01B7TMagaM8xBnYXLSMwUDAG', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 148, 'output_tokens': 82}}, id='run-c5aa9a8f-7983-4a7f-9c1e-0c0055334ac1-0')],
 'summary': "Here is a summary of our conversation so far:\n\n- You introduced yourself as Bob and said you like the Boston Celtics basketball team.\n- I acknowledged that it's nice to meet you, Bob, and noted that you had shared your name earlier in th

We can now resume having a conversation! Note that even though we only have the last two messages, we can still ask it questions about things mentioned earlier in the conversation (because we summarized those)

In [31]:
input_message = HumanMessage(content="what's my name?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

In our conversation so far, you introduced yourself as Bob. I acknowledged that earlier when you had shared your name.


In [32]:
input_message = HumanMessage(content="what NFL team do you think I like?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

what NFL team do you think I like?
================================== Ai Message ==================================

I don't actually have any information about what NFL team you might like. In our conversation so far, you've only mentioned that you're a fan of the Boston Celtics basketball team. I don't have any prior knowledge about your preferences for NFL teams. Unless you provide me with that information, I don't have a basis to guess which NFL team you might be a fan of.


In [33]:
input_message = HumanMessage(content="i like the patriots!")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

i like the patriots!
================================== Ai Message ==================================

Okay, got it! Thanks for sharing that you're also a fan of the New England Patriots in the NFL. That makes sense, given your interest in other Boston sports teams like the Celtics. The Patriots have also had a very successful run over the past couple of decades, winning multiple Super Bowls. It's fun to follow winning franchises like the Celtics and Patriots. Do you have a favorite Patriots player or moment that stands out to you?
================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message 

---